In [0]:
%pip install kaggle


In [0]:
dbutils.library.restartPython()


In [0]:
import os

In [0]:
os.environ["KAGGLE_USERNAME"] = dbutils.secrets.get(
    scope="kaggle",
    key="username"
)

os.environ["KAGGLE_KEY"] = dbutils.secrets.get(
    scope="kaggle",
    key="key"
)

assert os.getenv("KAGGLE_USERNAME"), "Defina KAGGLE_USERNAME"
assert os.getenv("KAGGLE_KEY"), "Defina KAGGLE_KEY"

In [0]:
import datetime

DATASET = "teocalvo/teomewhy-loyalty-system"
run_ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# onde o kaggle CLI vai baixar + unzip
download_path = f"/Volumes/raw/sistema_pontos/full_load/_staging/{run_ts}"

# destino final (parquet por tabela)
FULL_BASE = "/Volumes/raw/sistema_pontos/full_load"


In [0]:
!mkdir -p "{download_path}"
!kaggle datasets download -d "{DATASET}" -p "{download_path}" --unzip
!ls -lah "{download_path}" | head


In [0]:
import os, glob
from pyspark.sql import functions as F

csv_files = glob.glob(f"{download_path}/*.csv")
print("CSVs:", [os.path.basename(x) for x in csv_files])

for f in csv_files:
    name = os.path.splitext(os.path.basename(f))[0]
    out_path = f"{FULL_BASE}/{name}"

    df = (
        spark.read
        .option("header", True)
        .option("sep", ";")          
        .option("inferSchema", True) 
        .csv(f)
        .withColumn("_ingestion_ts", F.current_timestamp())
    )

    (
        df.coalesce(1)
        .write
        .mode("overwrite")
        .format("parquet")
        .save(out_path)
    )

    print("✅ full_load gravado:", out_path)


In [0]:
!rm -rf "{download_path}"
print("🧹 staging removido:", download_path)
